# Naive Bayes (Optional)
##### MNIST - Hand Written Digits

### Importing Libraries

In [11]:
from torchvision import datasets
import numpy as np
from scipy.ndimage import gaussian_filter

<hr>

### Loading Data

In [54]:
train_data = datasets.MNIST('./data', train=True, download=True)
train_images = np.array(train_data.train_data)/255
train_labels = np.array(train_data.train_labels)

test_data  = datasets.MNIST('./data', train=False, download=True)
test_images = np.array(test_data.test_data)/255
test_labels = np.array(test_data.test_labels)


Using downloaded and verified file: ./data\MNIST\raw\train-images-idx3-ubyte.gz
Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw
Using downloaded and verified file: ./data\MNIST\raw\train-labels-idx1-ubyte.gz
Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw
Using downloaded and verified file: ./data\MNIST\raw\t10k-images-idx3-ubyte.gz
Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw
Using downloaded and verified file: ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!


<hr>

### Training Model

In [50]:
def guassianFilter(train_images1):
    new_trained = np.zeros((len(train_images1),784))
    for i in range(train_images1.shape[0]):
        new_trained[i] = gaussian_filter(train_images1[i],sigma=1).flatten()
        new_trained[i] = new_trained[i]/np.max(new_trained[i])
    return new_trained

train_images = guassianFilter(train_images)
cpt_x = np.zeros((10,len(train_images[0])))
cpt_cov = np.zeros((10,len(train_images[0]),len(train_images[0])))
cpt_det_cov = np.zeros(10)
tmp = 0.1 * np.identity(784)

for j in range(10):
    target = train_images[train_labels==j]
    tmp2 = np.cov(target,rowvar=False)+tmp
    cpt_det_cov[j] = np.linalg.det(tmp2)
    cpt_cov[j,:]= np.linalg.inv(tmp2)
    cpt_x[j,:]= np.mean(target,axis=0)
num_count = np.zeros(10)
for i in range(10):
    num_count[i] = np.sum(train_labels == i)
cpt_y = num_count / np.sum(num_count)
print('Model trained!')

Model trained!


### Evaluating Model

In [53]:
test_images = guassianFilter(test_images)
n = len(test_images)
err = 0
for i in range(n):
    maximum = -100000000
    j_result = 0
    for j in range(10):
        tmp = np.matmul((test_images[i,:]-cpt_x[j,:]).T,cpt_cov[j])
        tmp = (1/2)*np.matmul(tmp, test_images[i,:]-cpt_x[j,:])
        tmp = tmp + (1/2)*np.log(cpt_det_cov[j]+1e-150)
        prob = np.log(cpt_y[j])-tmp
        if maximum<prob or j==0 and prob!='inf':
            j_result = j
            maximum = prob
    if j_result != test_labels[i]:
        err=err+1
print('Accuracy = ', 1-err/n)

Accuracy =  0.9627
